In [1]:
from pyspark.sql import SparkSession
from pyspark import SparkConf
from pyspark.sql import functions as f
from pyspark.sql import types as t
from pyspark.sql.window import Window

sparkSession = SparkSession.builder.config(conf=SparkConf() \
                                    .setAppName('Windows') \
                                    .setMaster('local[2]')).enableHiveSupport() \
                                    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/01/02 14:13:57 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
23/01/02 14:13:57 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
23/01/02 14:13:57 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.
23/01/02 14:13:57 WARN Utils: Service 'SparkUI' could not bind on port 4043. Attempting port 4044.
23/01/02 14:13:57 WARN Utils: Service 'SparkUI' could not bind on port 4044. Attempting port 4045.
23/01/02 14:13:57 WARN Utils: Service 'SparkUI' could not bind on port 4045. Attempting port 4046.
23/01/02 14:13:57 WARN Utils: Service 'SparkUI' could not bind on port 4046. Attempting port 4047.
23/01/02 14:13:57 WARN Utils: Service 'SparkUI' could not bind on port 4047. Attempting port 4048.
23/01/02 14:13:57 WARN Utils: Service 'SparkUI' could not bind on port 4048. A

#### PySpark Window Functions

PySpark Window functions are used to calculate results such as the rank, row number e.t.c over a range of input rows. In this article, I’ve explained the concept of window functions, syntax, and finally how to use them with PySpark SQL and PySpark DataFrame API. These come in handy when we need to make aggregate operations in a specific window frame on DataFrame columns.

When possible try to leverage standard library as they are little bit more compile-time safety, handles null and perform better when compared to UDF’s. If your application is critical on performance try to avoid using custom UDF at all costs as these are not guarantee on performance.

Window Functions

PySpark Window functions operate on a group of rows (like frame, partition) and return a single value for every input row. PySpark SQL supports three kinds of window functions:

1. ranking functions
2. analytic functions
3. aggregate functions

The below table defines Ranking and Analytic functions and for aggregate functions, we can use any existing aggregate functions as a window function.

To perform an operation on a group first, we need to partition the data using Window.partitionBy() , and for row number and rank function we need to additionally order by on partition data using orderBy clause.

Click on each link to know more about these functions along with the Scala examples.

In [3]:
simpleData = (("James", "Sales", 3000), \
    ("Michael", "Sales", 4600),  \
    ("Robert", "Sales", 4100),   \
    ("Maria", "Finance", 3000),  \
    ("James", "Sales", 3000),    \
    ("Scott", "Finance", 3300),  \
    ("Jen", "Finance", 3900),    \
    ("Jeff", "Marketing", 3000), \
    ("Kumar", "Marketing", 2000),\
    ("Saif", "Sales", 4100) \
  )
 
columns= ["employee_name", "department", "salary"]
df = sparkSession.createDataFrame(data = simpleData, schema = columns)
df.printSchema()
df.show(truncate=False)


root
 |-- employee_name: string (nullable = true)
 |-- department: string (nullable = true)
 |-- salary: long (nullable = true)



+-------------+----------+------+
|employee_name|department|salary|
+-------------+----------+------+
|James        |Sales     |3000  |
|Michael      |Sales     |4600  |
|Robert       |Sales     |4100  |
|Maria        |Finance   |3000  |
|James        |Sales     |3000  |
|Scott        |Finance   |3300  |
|Jen          |Finance   |3900  |
|Jeff         |Marketing |3000  |
|Kumar        |Marketing |2000  |
|Saif         |Sales     |4100  |
+-------------+----------+------+



In [23]:
data = sparkSession.read.csv('/Datasets/emp/train.csv', inferSchema=True, header=True).select(
 'Department',
 'EmployeeNumber',
 'MonthlyIncome',
 'Over18',
 'YearsSinceLastPromotion',
 'YearsWithCurrManager')
data.show(5)

+--------------------+--------------+-------------+------+-----------------------+--------------------+
|          Department|EmployeeNumber|MonthlyIncome|Over18|YearsSinceLastPromotion|YearsWithCurrManager|
+--------------------+--------------+-------------+------+-----------------------+--------------------+
|               Sales|             1|         5993|     Y|                      0|                   5|
|Research & Develo...|             2|         5130|     Y|                      1|                   7|
|Research & Develo...|             4|         2090|     Y|                      0|                   0|
|Research & Develo...|             5|         2909|     Y|                      3|                   0|
|Research & Develo...|             7|         3468|     Y|                      2|                   2|
+--------------------+--------------+-------------+------+-----------------------+--------------------+
only showing top 5 rows



#### PySpark Window Aggregate Functions

In this section, I will explain how to calculate sum, min, max for each department using PySpark SQL Aggregate window functions and WindowSpec. When working with Aggregate functions, we don’t need to use order by clause. 

In [9]:
df.show()

+-------------+----------+------+
|employee_name|department|salary|
+-------------+----------+------+
|        James|     Sales|  3000|
|      Michael|     Sales|  4600|
|       Robert|     Sales|  4100|
|        Maria|   Finance|  3000|
|        James|     Sales|  3000|
|        Scott|   Finance|  3300|
|          Jen|   Finance|  3900|
|         Jeff| Marketing|  3000|
|        Kumar| Marketing|  2000|
|         Saif|     Sales|  4100|
+-------------+----------+------+



In [11]:
df.groupBy('department').agg(f.sum(f.col('salary')).alias('salary_sum'), \
                             f.avg(f.col('salary')).alias('salary_avg'), \
                            f.max(f.col('salary')).alias('salary_max'), \
                            f.collect_list(f.col('salary')).alias('list')).show()

+----------+----------+----------+----------+--------------------+
|department|salary_sum|salary_avg|salary_max|                list|
+----------+----------+----------+----------+--------------------+
|     Sales|     18800|    3760.0|      4600|[3000, 4600, 4100...|
|   Finance|     10200|    3400.0|      3900|  [3000, 3300, 3900]|
| Marketing|      5000|    2500.0|      3000|        [3000, 2000]|
+----------+----------+----------+----------+--------------------+



In [31]:
window1 = Window.partitionBy('department')
df.withColumn('list1', f.collect_list(f.col('salary')).over(window1)) \
    .withColumn('salary_sum', f.sum(f.col('salary')).over(window1)) \
    .withColumn('salary_avg', f.avg(f.col('salary')).over(window1)) \
    .withColumn('salary_max', f.max(f.col('salary')).over(window1)).select('*').show()

+-------------+----------+------+--------------------+----------+----------+----------+
|employee_name|department|salary|               list1|salary_sum|salary_avg|salary_max|
+-------------+----------+------+--------------------+----------+----------+----------+
|        Maria|   Finance|  3000|  [3000, 3300, 3900]|     10200|    3400.0|      3900|
|        Scott|   Finance|  3300|  [3000, 3300, 3900]|     10200|    3400.0|      3900|
|          Jen|   Finance|  3900|  [3000, 3300, 3900]|     10200|    3400.0|      3900|
|         Jeff| Marketing|  3000|        [3000, 2000]|      5000|    2500.0|      3000|
|        Kumar| Marketing|  2000|        [3000, 2000]|      5000|    2500.0|      3000|
|        James|     Sales|  3000|[3000, 4600, 4100...|     18800|    3760.0|      4600|
|      Michael|     Sales|  4600|[3000, 4600, 4100...|     18800|    3760.0|      4600|
|       Robert|     Sales|  4100|[3000, 4600, 4100...|     18800|    3760.0|      4600|
|        James|     Sales|  3000

In [34]:
window1 = Window.partitionBy('department').orderBy('salary')

df.withColumn('list1', f.collect_list(f.col('salary')).over(window1)) \
    .withColumn('salary_sum', f.sum(f.col('salary')).over(window1)) \
    .withColumn('salary_avg', f.avg(f.col('salary')).over(window1)) \
    .withColumn('salary_max', f.max(f.col('salary')).over(window1)).select('*').show()

+-------------+----------+------+--------------------+----------+----------+----------+
|employee_name|department|salary|               list1|salary_sum|salary_avg|salary_max|
+-------------+----------+------+--------------------+----------+----------+----------+
|        Maria|   Finance|  3000|              [3000]|      3000|    3000.0|      3000|
|        Scott|   Finance|  3300|        [3000, 3300]|      6300|    3150.0|      3300|
|          Jen|   Finance|  3900|  [3000, 3300, 3900]|     10200|    3400.0|      3900|
|        Kumar| Marketing|  2000|              [2000]|      2000|    2000.0|      2000|
|         Jeff| Marketing|  3000|        [2000, 3000]|      5000|    2500.0|      3000|
|        James|     Sales|  3000|        [3000, 3000]|      6000|    3000.0|      3000|
|        James|     Sales|  3000|        [3000, 3000]|      6000|    3000.0|      3000|
|       Robert|     Sales|  4100|[3000, 3000, 4100...|     14200|    3550.0|      4100|
|         Saif|     Sales|  4100

In [35]:
window1 = Window.partitionBy('department').orderBy('salary') \
            .rowsBetween(Window.unboundedPreceding, Window.currentRow)

df.withColumn('list1', f.collect_list(f.col('salary')).over(window1)) \
    .withColumn('salary_sum', f.sum(f.col('salary')).over(window1)) \
    .withColumn('salary_avg', f.avg(f.col('salary')).over(window1)) \
    .withColumn('salary_max', f.max(f.col('salary')).over(window1)).select('*').show()

+-------------+----------+------+--------------------+----------+------------------+----------+
|employee_name|department|salary|               list1|salary_sum|        salary_avg|salary_max|
+-------------+----------+------+--------------------+----------+------------------+----------+
|        Maria|   Finance|  3000|              [3000]|      3000|            3000.0|      3000|
|        Scott|   Finance|  3300|        [3000, 3300]|      6300|            3150.0|      3300|
|          Jen|   Finance|  3900|  [3000, 3300, 3900]|     10200|            3400.0|      3900|
|        Kumar| Marketing|  2000|              [2000]|      2000|            2000.0|      2000|
|         Jeff| Marketing|  3000|        [2000, 3000]|      5000|            2500.0|      3000|
|        James|     Sales|  3000|              [3000]|      3000|            3000.0|      3000|
|        James|     Sales|  3000|        [3000, 3000]|      6000|            3000.0|      3000|
|       Robert|     Sales|  4100|  [3000

In [36]:
window1 = Window.partitionBy('department').orderBy('salary') \
            .rowsBetween(-1, Window.currentRow)

df.withColumn('list1', f.collect_list(f.col('salary')).over(window1)) \
    .withColumn('salary_sum', f.sum(f.col('salary')).over(window1)) \
    .withColumn('salary_avg', f.avg(f.col('salary')).over(window1)) \
    .withColumn('salary_max', f.max(f.col('salary')).over(window1)).select('*').show()

+-------------+----------+------+------------+----------+----------+----------+
|employee_name|department|salary|       list1|salary_sum|salary_avg|salary_max|
+-------------+----------+------+------------+----------+----------+----------+
|        Maria|   Finance|  3000|      [3000]|      3000|    3000.0|      3000|
|        Scott|   Finance|  3300|[3000, 3300]|      6300|    3150.0|      3300|
|          Jen|   Finance|  3900|[3300, 3900]|      7200|    3600.0|      3900|
|        Kumar| Marketing|  2000|      [2000]|      2000|    2000.0|      2000|
|         Jeff| Marketing|  3000|[2000, 3000]|      5000|    2500.0|      3000|
|        James|     Sales|  3000|      [3000]|      3000|    3000.0|      3000|
|        James|     Sales|  3000|[3000, 3000]|      6000|    3000.0|      3000|
|       Robert|     Sales|  4100|[3000, 4100]|      7100|    3550.0|      4100|
|         Saif|     Sales|  4100|[4100, 4100]|      8200|    4100.0|      4100|
|      Michael|     Sales|  4600|[4100, 

In [39]:
data.show(3)

+--------------------+--------------+-------------+------+-----------------------+--------------------+
|          Department|EmployeeNumber|MonthlyIncome|Over18|YearsSinceLastPromotion|YearsWithCurrManager|
+--------------------+--------------+-------------+------+-----------------------+--------------------+
|               Sales|             1|         5993|     Y|                      0|                   5|
|Research & Develo...|             2|         5130|     Y|                      1|                   7|
|Research & Develo...|             4|         2090|     Y|                      0|                   0|
+--------------------+--------------+-------------+------+-----------------------+--------------------+
only showing top 3 rows



In [42]:
data.groupBy('Department').agg(f.collect_list(f.col('MonthlyIncome')).alias('list1') \
                              ,f.sum(f.col('MonthlyIncome')).alias('salary_sum') \
                              ,f.avg(f.col('MonthlyIncome')).alias('salary_avg') \
                              ,f.max(f.col('MonthlyIncome')).alias('salary_max')).show()
                                

+--------------------+--------------------+----------+-----------------+----------+
|          Department|               list1|salary_sum|       salary_avg|salary_max|
+--------------------+--------------------+----------+-----------------+----------+
|               Sales|[5993, 15427, 340...|   2241789|7027.551724137931|     19847|
|Research & Develo...|[5130, 2090, 2909...|   4488018|6402.308131241084|     19999|
|     Human Resources|[5021, 2073, 1884...|    280443|7380.078947368421|     19717|
+--------------------+--------------------+----------+-----------------+----------+



In [48]:
window1 = Window.partitionBy('Department')
data.withColumn('list1', f.collect_list(f.col('MonthlyIncome')).over(window1)) \
    .withColumn('salary_sum', f.sum(f.col('MonthlyIncome')).over(window1)) \
    .withColumn('salary_avg', f.avg(f.col('MonthlyIncome')).over(window1)) \
    .withColumn('salary_max', f.max(f.col('MonthlyIncome')).over(window1)).select('Department', \
    'list1', 'salary_sum', 'salary_avg', 'salary_max').show()

+---------------+--------------------+----------+-----------------+----------+
|     Department|               list1|salary_sum|       salary_avg|salary_max|
+---------------+--------------------+----------+-----------------+----------+
|Human Resources|[5021, 2073, 1884...|    280443|7380.078947368421|     19717|
|Human Resources|[5021, 2073, 1884...|    280443|7380.078947368421|     19717|
|Human Resources|[5021, 2073, 1884...|    280443|7380.078947368421|     19717|
|Human Resources|[5021, 2073, 1884...|    280443|7380.078947368421|     19717|
|Human Resources|[5021, 2073, 1884...|    280443|7380.078947368421|     19717|
|Human Resources|[5021, 2073, 1884...|    280443|7380.078947368421|     19717|
|Human Resources|[5021, 2073, 1884...|    280443|7380.078947368421|     19717|
|Human Resources|[5021, 2073, 1884...|    280443|7380.078947368421|     19717|
|Human Resources|[5021, 2073, 1884...|    280443|7380.078947368421|     19717|
|Human Resources|[5021, 2073, 1884...|    280443|738

In [49]:
window1 = Window.partitionBy('Department').orderBy('MonthlyIncome')
data.withColumn('list1', f.collect_list(f.col('MonthlyIncome')).over(window1)) \
    .withColumn('salary_sum', f.sum(f.col('MonthlyIncome')).over(window1)) \
    .withColumn('salary_avg', f.avg(f.col('MonthlyIncome')).over(window1)) \
    .withColumn('salary_max', f.max(f.col('MonthlyIncome')).over(window1)).select('Department', \
    'list1', 'salary_sum', 'salary_avg', 'salary_max').show()

+---------------+--------------------+----------+------------------+----------+
|     Department|               list1|salary_sum|        salary_avg|salary_max|
+---------------+--------------------+----------+------------------+----------+
|Human Resources|              [2073]|      2073|            2073.0|      2073|
|Human Resources|        [2073, 2109]|      4182|            2091.0|      2109|
|Human Resources|  [2073, 2109, 2143]|      6325|2108.3333333333335|      2143|
|Human Resources|[2073, 2109, 2143...|      8502|            2125.5|      2177|
|Human Resources|[2073, 2109, 2143...|     10769|            2153.8|      2267|
|Human Resources|[2073, 2109, 2143...|     13046|2174.3333333333335|      2277|
|Human Resources|[2073, 2109, 2143...|     15388| 2198.285714285714|      2342|
|Human Resources|[2073, 2109, 2143...|     17952|            2244.0|      2564|
|Human Resources|[2073, 2109, 2143...|     20648| 2294.222222222222|      2696|
|Human Resources|[2073, 2109, 2143...|  

In [50]:
window1 = Window.partitionBy('Department').orderBy('MonthlyIncome').rowsBetween(\
                Window.unboundedPreceding, Window.currentRow)
data.withColumn('list1', f.collect_list(f.col('MonthlyIncome')).over(window1)) \
    .withColumn('salary_sum', f.sum(f.col('MonthlyIncome')).over(window1)) \
    .withColumn('salary_avg', f.avg(f.col('MonthlyIncome')).over(window1)) \
    .withColumn('salary_max', f.max(f.col('MonthlyIncome')).over(window1)).select('Department', \
    'list1', 'salary_sum', 'salary_avg', 'salary_max').show()

+---------------+--------------------+----------+------------------+----------+
|     Department|               list1|salary_sum|        salary_avg|salary_max|
+---------------+--------------------+----------+------------------+----------+
|Human Resources|              [2073]|      2073|            2073.0|      2073|
|Human Resources|        [2073, 2109]|      4182|            2091.0|      2109|
|Human Resources|  [2073, 2109, 2143]|      6325|2108.3333333333335|      2143|
|Human Resources|[2073, 2109, 2143...|      8502|            2125.5|      2177|
|Human Resources|[2073, 2109, 2143...|     10769|            2153.8|      2267|
|Human Resources|[2073, 2109, 2143...|     13046|2174.3333333333335|      2277|
|Human Resources|[2073, 2109, 2143...|     15388| 2198.285714285714|      2342|
|Human Resources|[2073, 2109, 2143...|     17952|            2244.0|      2564|
|Human Resources|[2073, 2109, 2143...|     20648| 2294.222222222222|      2696|
|Human Resources|[2073, 2109, 2143...|  

#### PySpark Window Ranking functions

row_number Window Function

row_number() window function is used to give the sequential row number starting from 1 to the result of each window partition. 

In [51]:
df.show(5)

+-------------+----------+------+
|employee_name|department|salary|
+-------------+----------+------+
|        James|     Sales|  3000|
|      Michael|     Sales|  4600|
|       Robert|     Sales|  4100|
|        Maria|   Finance|  3000|
|        James|     Sales|  3000|
+-------------+----------+------+
only showing top 5 rows



In [52]:
window = Window.partitionBy('department').orderBy('salary')

df.withColumn('row_number', f.row_number().over(window)).show()

+-------------+----------+------+----------+
|employee_name|department|salary|row_number|
+-------------+----------+------+----------+
|        Maria|   Finance|  3000|         1|
|        Scott|   Finance|  3300|         2|
|          Jen|   Finance|  3900|         3|
|        Kumar| Marketing|  2000|         1|
|         Jeff| Marketing|  3000|         2|
|        James|     Sales|  3000|         1|
|        James|     Sales|  3000|         2|
|       Robert|     Sales|  4100|         3|
|         Saif|     Sales|  4100|         4|
|      Michael|     Sales|  4600|         5|
+-------------+----------+------+----------+



In [54]:
window = Window.partitionBy('Department').orderBy('MonthlyIncome')

data.withColumn('row_number', f.row_number().over(window)).show(5)

+---------------+--------------+-------------+------+-----------------------+--------------------+----------+
|     Department|EmployeeNumber|MonthlyIncome|Over18|YearsSinceLastPromotion|YearsWithCurrManager|row_number|
+---------------+--------------+-------------+------+-----------------------+--------------------+----------+
|Human Resources|           133|         2073|     Y|                      0|                   2|         1|
|Human Resources|          1461|         2109|     Y|                      0|                   2|         2|
|Human Resources|           829|         2143|     Y|                      0|                   4|         3|
|Human Resources|          1049|         2177|     Y|                      0|                   4|         4|
|Human Resources|           321|         2267|     Y|                      2|                   2|         5|
+---------------+--------------+-------------+------+-----------------------+--------------------+----------+
only showi

### rank Window Function

rank() window function is used to provide a rank to the result within a window partition. This function leaves gaps in rank when there are ties.

In [56]:
window = Window.partitionBy('Department').orderBy('MonthlyIncome')
data.withColumn('rank', f.rank().over(window)).show(5)

+---------------+--------------+-------------+------+-----------------------+--------------------+----+
|     Department|EmployeeNumber|MonthlyIncome|Over18|YearsSinceLastPromotion|YearsWithCurrManager|rank|
+---------------+--------------+-------------+------+-----------------------+--------------------+----+
|Human Resources|           133|         2073|     Y|                      0|                   2|   1|
|Human Resources|          1461|         2109|     Y|                      0|                   2|   2|
|Human Resources|           829|         2143|     Y|                      0|                   4|   3|
|Human Resources|          1049|         2177|     Y|                      0|                   4|   4|
|Human Resources|           321|         2267|     Y|                      2|                   2|   5|
+---------------+--------------+-------------+------+-----------------------+--------------------+----+
only showing top 5 rows



In [58]:
window = Window.partitionBy('department').orderBy('salary')
df.withColumn('rank', f.rank().over(window)).show()

+-------------+----------+------+----+
|employee_name|department|salary|rank|
+-------------+----------+------+----+
|        Maria|   Finance|  3000|   1|
|        Scott|   Finance|  3300|   2|
|          Jen|   Finance|  3900|   3|
|        Kumar| Marketing|  2000|   1|
|         Jeff| Marketing|  3000|   2|
|        James|     Sales|  3000|   1|
|        James|     Sales|  3000|   1|
|       Robert|     Sales|  4100|   3|
|         Saif|     Sales|  4100|   3|
|      Michael|     Sales|  4600|   5|
+-------------+----------+------+----+



#### dense_rank Window Function

dense_rank() window function is used to get the result with rank of rows within a window partition without any gaps. This is similar to rank() function difference being rank function leaves gaps in rank when there are ties.

In [59]:
window = Window.partitionBy('Department').orderBy('MonthlyIncome')
data.withColumn('dense_rank', f.dense_rank().over(window)).show()

+---------------+--------------+-------------+------+-----------------------+--------------------+----------+
|     Department|EmployeeNumber|MonthlyIncome|Over18|YearsSinceLastPromotion|YearsWithCurrManager|dense_rank|
+---------------+--------------+-------------+------+-----------------------+--------------------+----------+
|Human Resources|           133|         2073|     Y|                      0|                   2|         1|
|Human Resources|          1461|         2109|     Y|                      0|                   2|         2|
|Human Resources|           829|         2143|     Y|                      0|                   4|         3|
|Human Resources|          1049|         2177|     Y|                      0|                   4|         4|
|Human Resources|           321|         2267|     Y|                      2|                   2|         5|
|Human Resources|           910|         2277|     Y|                      0|                   3|         6|
|Human Res

In [60]:
window = Window.partitionBy('department').orderBy('salary')
df.withColumn('dense_rank', f.dense_rank().over(window)).show()

+-------------+----------+------+----------+
|employee_name|department|salary|dense_rank|
+-------------+----------+------+----------+
|        Maria|   Finance|  3000|         1|
|        Scott|   Finance|  3300|         2|
|          Jen|   Finance|  3900|         3|
|        Kumar| Marketing|  2000|         1|
|         Jeff| Marketing|  3000|         2|
|        James|     Sales|  3000|         1|
|        James|     Sales|  3000|         1|
|       Robert|     Sales|  4100|         2|
|         Saif|     Sales|  4100|         2|
|      Michael|     Sales|  4600|         3|
+-------------+----------+------+----------+



#### percent_rank Window Function 

In [61]:
window = Window.partitionBy('department').orderBy('salary')
df.withColumn('percent_rank', f.percent_rank().over(window)).show()

+-------------+----------+------+------------+
|employee_name|department|salary|percent_rank|
+-------------+----------+------+------------+
|        Maria|   Finance|  3000|         0.0|
|        Scott|   Finance|  3300|         0.5|
|          Jen|   Finance|  3900|         1.0|
|        Kumar| Marketing|  2000|         0.0|
|         Jeff| Marketing|  3000|         1.0|
|        James|     Sales|  3000|         0.0|
|        James|     Sales|  3000|         0.0|
|       Robert|     Sales|  4100|         0.5|
|         Saif|     Sales|  4100|         0.5|
|      Michael|     Sales|  4600|         1.0|
+-------------+----------+------+------------+



In [62]:
window = Window.partitionBy('Department').orderBy('MonthlyIncome')
data.withColumn('percent_rank', f.percent_rank().over(window)).show()

+---------------+--------------+-------------+------+-----------------------+--------------------+-------------------+
|     Department|EmployeeNumber|MonthlyIncome|Over18|YearsSinceLastPromotion|YearsWithCurrManager|       percent_rank|
+---------------+--------------+-------------+------+-----------------------+--------------------+-------------------+
|Human Resources|           133|         2073|     Y|                      0|                   2|                0.0|
|Human Resources|          1461|         2109|     Y|                      0|                   2|0.02702702702702703|
|Human Resources|           829|         2143|     Y|                      0|                   4|0.05405405405405406|
|Human Resources|          1049|         2177|     Y|                      0|                   4|0.08108108108108109|
|Human Resources|           321|         2267|     Y|                      2|                   2|0.10810810810810811|
|Human Resources|           910|         2277|  

#### ntile Window Function

ntile() window function returns the relative rank of result rows within a window partition. In below example we have used 2 as an argument to ntile hence it returns ranking between 2 values (1 and 2) 

In [64]:
window = Window.partitionBy('department').orderBy('salary')
df.withColumn('ntile', f.ntile(2).over(window)).show()

+-------------+----------+------+-----+
|employee_name|department|salary|ntile|
+-------------+----------+------+-----+
|        Maria|   Finance|  3000|    1|
|        Scott|   Finance|  3300|    1|
|          Jen|   Finance|  3900|    2|
|        Kumar| Marketing|  2000|    1|
|         Jeff| Marketing|  3000|    2|
|        James|     Sales|  3000|    1|
|        James|     Sales|  3000|    1|
|       Robert|     Sales|  4100|    1|
|         Saif|     Sales|  4100|    2|
|      Michael|     Sales|  4600|    2|
+-------------+----------+------+-----+



In [65]:
window = Window.partitionBy('department').orderBy('salary')
df.withColumn('ntile', f.ntile(3).over(window)).show()

+-------------+----------+------+-----+
|employee_name|department|salary|ntile|
+-------------+----------+------+-----+
|        Maria|   Finance|  3000|    1|
|        Scott|   Finance|  3300|    2|
|          Jen|   Finance|  3900|    3|
|        Kumar| Marketing|  2000|    1|
|         Jeff| Marketing|  3000|    2|
|        James|     Sales|  3000|    1|
|        James|     Sales|  3000|    1|
|       Robert|     Sales|  4100|    2|
|         Saif|     Sales|  4100|    2|
|      Michael|     Sales|  4600|    3|
+-------------+----------+------+-----+



In [66]:
window = Window.partitionBy('department').orderBy('salary')
df.withColumn('ntile', f.ntile(4).over(window)).show()

+-------------+----------+------+-----+
|employee_name|department|salary|ntile|
+-------------+----------+------+-----+
|        Maria|   Finance|  3000|    1|
|        Scott|   Finance|  3300|    2|
|          Jen|   Finance|  3900|    3|
|        Kumar| Marketing|  2000|    1|
|         Jeff| Marketing|  3000|    2|
|        James|     Sales|  3000|    1|
|        James|     Sales|  3000|    1|
|       Robert|     Sales|  4100|    2|
|         Saif|     Sales|  4100|    3|
|      Michael|     Sales|  4600|    4|
+-------------+----------+------+-----+



In [67]:
window = Window.partitionBy('Department').orderBy('MonthlyIncome')
data.withColumn('ntile', f.ntile(2).over(window)).show()

+---------------+--------------+-------------+------+-----------------------+--------------------+-----+
|     Department|EmployeeNumber|MonthlyIncome|Over18|YearsSinceLastPromotion|YearsWithCurrManager|ntile|
+---------------+--------------+-------------+------+-----------------------+--------------------+-----+
|Human Resources|           133|         2073|     Y|                      0|                   2|    1|
|Human Resources|          1461|         2109|     Y|                      0|                   2|    1|
|Human Resources|           829|         2143|     Y|                      0|                   4|    1|
|Human Resources|          1049|         2177|     Y|                      0|                   4|    1|
|Human Resources|           321|         2267|     Y|                      2|                   2|    1|
|Human Resources|           910|         2277|     Y|                      0|                   3|    1|
|Human Resources|           878|         2342|     Y|  

In [68]:
window = Window.partitionBy('Department').orderBy('MonthlyIncome')
data.withColumn('ntile', f.ntile(4).over(window)).show()

+---------------+--------------+-------------+------+-----------------------+--------------------+-----+
|     Department|EmployeeNumber|MonthlyIncome|Over18|YearsSinceLastPromotion|YearsWithCurrManager|ntile|
+---------------+--------------+-------------+------+-----------------------+--------------------+-----+
|Human Resources|           133|         2073|     Y|                      0|                   2|    1|
|Human Resources|          1461|         2109|     Y|                      0|                   2|    1|
|Human Resources|           829|         2143|     Y|                      0|                   4|    1|
|Human Resources|          1049|         2177|     Y|                      0|                   4|    1|
|Human Resources|           321|         2267|     Y|                      2|                   2|    1|
|Human Resources|           910|         2277|     Y|                      0|                   3|    1|
|Human Resources|           878|         2342|     Y|  

In [69]:
window = Window.partitionBy('Department').orderBy('MonthlyIncome')
data.withColumn('ntile', f.ntile(10).over(window)).show()

+---------------+--------------+-------------+------+-----------------------+--------------------+-----+
|     Department|EmployeeNumber|MonthlyIncome|Over18|YearsSinceLastPromotion|YearsWithCurrManager|ntile|
+---------------+--------------+-------------+------+-----------------------+--------------------+-----+
|Human Resources|           133|         2073|     Y|                      0|                   2|    1|
|Human Resources|          1461|         2109|     Y|                      0|                   2|    1|
|Human Resources|           829|         2143|     Y|                      0|                   4|    1|
|Human Resources|          1049|         2177|     Y|                      0|                   4|    1|
|Human Resources|           321|         2267|     Y|                      2|                   2|    2|
|Human Resources|           910|         2277|     Y|                      0|                   3|    2|
|Human Resources|           878|         2342|     Y|  

#### PySpark Window Analytic functions

#### cume_dist Window Function

cume_dist() window function is used to get the cumulative distribution of values within a window partition.

This is the same as the DENSE_RANK function in SQL. 

In [71]:
window = Window.partitionBy('department').orderBy('salary')
df.withColumn('cum_dist', f.cume_dist().over(window)).show()

+-------------+----------+------+------------------+
|employee_name|department|salary|          cum_dist|
+-------------+----------+------+------------------+
|        Maria|   Finance|  3000|0.3333333333333333|
|        Scott|   Finance|  3300|0.6666666666666666|
|          Jen|   Finance|  3900|               1.0|
|        Kumar| Marketing|  2000|               0.5|
|         Jeff| Marketing|  3000|               1.0|
|        James|     Sales|  3000|               0.4|
|        James|     Sales|  3000|               0.4|
|       Robert|     Sales|  4100|               0.8|
|         Saif|     Sales|  4100|               0.8|
|      Michael|     Sales|  4600|               1.0|
+-------------+----------+------+------------------+



In [74]:
window = Window.partitionBy('department').orderBy('salary')
df.withColumn('lead', f.lead(f.col('salary'), 1).over(window)).show()

+-------------+----------+------+----+
|employee_name|department|salary|lead|
+-------------+----------+------+----+
|        Maria|   Finance|  3000|3300|
|        Scott|   Finance|  3300|3900|
|          Jen|   Finance|  3900|null|
|        Kumar| Marketing|  2000|3000|
|         Jeff| Marketing|  3000|null|
|        James|     Sales|  3000|3000|
|        James|     Sales|  3000|4100|
|       Robert|     Sales|  4100|4100|
|         Saif|     Sales|  4100|4600|
|      Michael|     Sales|  4600|null|
+-------------+----------+------+----+



In [75]:
window = Window.partitionBy('department').orderBy('salary')
df.withColumn('lead', f.lead(f.col('salary'), 2).over(window)).show()

+-------------+----------+------+----+
|employee_name|department|salary|lead|
+-------------+----------+------+----+
|        Maria|   Finance|  3000|3900|
|        Scott|   Finance|  3300|null|
|          Jen|   Finance|  3900|null|
|        Kumar| Marketing|  2000|null|
|         Jeff| Marketing|  3000|null|
|        James|     Sales|  3000|4100|
|        James|     Sales|  3000|4100|
|       Robert|     Sales|  4100|4600|
|         Saif|     Sales|  4100|null|
|      Michael|     Sales|  4600|null|
+-------------+----------+------+----+



In [76]:
window = Window.partitionBy('department').orderBy('salary')
df.withColumn('lead', f.lag(f.col('salary'), 1).over(window)).show()

+-------------+----------+------+----+
|employee_name|department|salary|lead|
+-------------+----------+------+----+
|        Maria|   Finance|  3000|null|
|        Scott|   Finance|  3300|3000|
|          Jen|   Finance|  3900|3300|
|        Kumar| Marketing|  2000|null|
|         Jeff| Marketing|  3000|2000|
|        James|     Sales|  3000|null|
|        James|     Sales|  3000|3000|
|       Robert|     Sales|  4100|3000|
|         Saif|     Sales|  4100|4100|
|      Michael|     Sales|  4600|4100|
+-------------+----------+------+----+



In [77]:
window = Window.partitionBy('department').orderBy('salary')
df.withColumn('lead', f.lead(f.col('salary'), 2).over(window)).show()

+-------------+----------+------+----+
|employee_name|department|salary|lead|
+-------------+----------+------+----+
|        Maria|   Finance|  3000|3900|
|        Scott|   Finance|  3300|null|
|          Jen|   Finance|  3900|null|
|        Kumar| Marketing|  2000|null|
|         Jeff| Marketing|  3000|null|
|        James|     Sales|  3000|4100|
|        James|     Sales|  3000|4100|
|       Robert|     Sales|  4100|4600|
|         Saif|     Sales|  4100|null|
|      Michael|     Sales|  4600|null|
+-------------+----------+------+----+



In [78]:
window = Window.partitionBy('Department').orderBy('MonthlyIncome')
data.withColumn('lead', f.lead(f.col('MonthlyIncome'),1).over(window)).show()

+---------------+--------------+-------------+------+-----------------------+--------------------+----+
|     Department|EmployeeNumber|MonthlyIncome|Over18|YearsSinceLastPromotion|YearsWithCurrManager|lead|
+---------------+--------------+-------------+------+-----------------------+--------------------+----+
|Human Resources|           133|         2073|     Y|                      0|                   2|2109|
|Human Resources|          1461|         2109|     Y|                      0|                   2|2143|
|Human Resources|           829|         2143|     Y|                      0|                   4|2177|
|Human Resources|          1049|         2177|     Y|                      0|                   4|2267|
|Human Resources|           321|         2267|     Y|                      2|                   2|2277|
|Human Resources|           910|         2277|     Y|                      0|                   3|2342|
|Human Resources|           878|         2342|     Y|           

In [79]:
window = Window.partitionBy('Department').orderBy('MonthlyIncome')
data.withColumn('lead', f.lead(f.col('MonthlyIncome'),2).over(window)).show()

+---------------+--------------+-------------+------+-----------------------+--------------------+----+
|     Department|EmployeeNumber|MonthlyIncome|Over18|YearsSinceLastPromotion|YearsWithCurrManager|lead|
+---------------+--------------+-------------+------+-----------------------+--------------------+----+
|Human Resources|           133|         2073|     Y|                      0|                   2|2143|
|Human Resources|          1461|         2109|     Y|                      0|                   2|2177|
|Human Resources|           829|         2143|     Y|                      0|                   4|2267|
|Human Resources|          1049|         2177|     Y|                      0|                   4|2277|
|Human Resources|           321|         2267|     Y|                      2|                   2|2342|
|Human Resources|           910|         2277|     Y|                      0|                   3|2564|
|Human Resources|           878|         2342|     Y|           

In [80]:
window = Window.partitionBy('Department').orderBy('MonthlyIncome')
data.withColumn('lag', f.lag(f.col('MonthlyIncome'),1).over(window)).show()

+---------------+--------------+-------------+------+-----------------------+--------------------+----+
|     Department|EmployeeNumber|MonthlyIncome|Over18|YearsSinceLastPromotion|YearsWithCurrManager| lag|
+---------------+--------------+-------------+------+-----------------------+--------------------+----+
|Human Resources|           133|         2073|     Y|                      0|                   2|null|
|Human Resources|          1461|         2109|     Y|                      0|                   2|2073|
|Human Resources|           829|         2143|     Y|                      0|                   4|2109|
|Human Resources|          1049|         2177|     Y|                      0|                   4|2143|
|Human Resources|           321|         2267|     Y|                      2|                   2|2177|
|Human Resources|           910|         2277|     Y|                      0|                   3|2267|
|Human Resources|           878|         2342|     Y|           

In [81]:
window = Window.partitionBy('Department').orderBy('MonthlyIncome')
data.withColumn('lag', f.lag(f.col('MonthlyIncome'),2).over(window)).show()

+---------------+--------------+-------------+------+-----------------------+--------------------+----+
|     Department|EmployeeNumber|MonthlyIncome|Over18|YearsSinceLastPromotion|YearsWithCurrManager| lag|
+---------------+--------------+-------------+------+-----------------------+--------------------+----+
|Human Resources|           133|         2073|     Y|                      0|                   2|null|
|Human Resources|          1461|         2109|     Y|                      0|                   2|null|
|Human Resources|           829|         2143|     Y|                      0|                   4|2073|
|Human Resources|          1049|         2177|     Y|                      0|                   4|2109|
|Human Resources|           321|         2267|     Y|                      2|                   2|2143|
|Human Resources|           910|         2277|     Y|                      0|                   3|2177|
|Human Resources|           878|         2342|     Y|           